<a href="https://colab.research.google.com/github/lluk4z/IF697-CienciaDeDados/blob/main/Projeto01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn

In [ ]:
from google.colab import files
import io
uploaded = files.upload()

Saving owid-covid-data.csv to owid-covid-data.csv


In [ ]:
df = pd.read_csv(io.StringIO(uploaded['owid-covid-data.csv'].decode('utf-8')))

In [ ]:
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,0.026,0.026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101998 entries, 0 to 101997
Data columns (total 60 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   iso_code                               101998 non-null  object 
 1   continent                              97230 non-null   object 
 2   location                               101998 non-null  object 
 3   date                                   101998 non-null  object 
 4   total_cases                            98192 non-null   float64
 5   new_cases                              98189 non-null   float64
 6   new_cases_smoothed                     97179 non-null   float64
 7   total_deaths                           87983 non-null   float64
 8   new_deaths                             88139 non-null   float64
 9   new_deaths_smoothed                    97179 non-null   float64
 10  total_cases_per_million                97670 non-null   

In [ ]:
df.isnull().sum()

iso_code                                      0
continent                                  4768
location                                      0
date                                          0
total_cases                                3806
new_cases                                  3809
new_cases_smoothed                         4819
total_deaths                              14015
new_deaths                                13859
new_deaths_smoothed                        4819
total_cases_per_million                    4328
new_cases_per_million                      4331
new_cases_smoothed_per_million             5336
total_deaths_per_million                  14524
new_deaths_per_million                    14368
new_deaths_smoothed_per_million            5336
reproduction_rate                         19771
icu_patients                              91741
icu_patients_per_million                  91741
hosp_patients                             89262
hosp_patients_per_million               

In [ ]:
df.isnull().sum().sum()

2480377

In [ ]:
df.describe()

,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
count,9.819200e+04,98189.000000,97179.000000,8.798300e+04,88139.000000,97179.000000,97670.000000,97667.000000,96662.000000,87474.000000,87630.000000,96662.000000,82227.000000,10257.000000,10257.000000,12736.000000,12736.000000,928.000000,928.000000,1591.000000,1591.000000,4.577300e+04,4.543500e+04,45435.000000,45773.000000,5.328800e+04,53288.000000,49803.000000,49173.000000,1.781400e+04,1.696400e+04,1.408800e+04,1.483900e+04,3.108000e+04,17814.000000,16964.000000,14088.000000,31080.000000,85561.000000,1.013420e+05,94735.000000,91054.000000,90032.000000,90551.000000,91423.000000,61663.000000,91428.000000,93811.000000,71488.000000,70445.000000,45943.000000,83260.000000,96855.000000,91566.000000,3571.000000
mean,1.105663e+06,6046.976790,6069.654773,2.967335e+04,145.884988,131.640184,13639.237674,76.294066,76.508641,299.677734,1.545376,1.393845,1.004412,1045.860973,25.888946,4543.305669,165.840489,260.635962,19.939103,3480.505454,106.654213,4.872986e+04,8.143243e+06,328.136050,2.161635,4.621628e+04,2.071310,0.088281,165.840475,3.545322e+07,1.962384e+07,1.094501e+07,7.037522e+05,3.398535e+05,26.232361,17.024026,10.977104,3312.322683,58.133985,1.243982e+08,387.069205,30.556026,8.785568,5.561554,19278.662350,13.419023,258.597273,7.933810,10.567042,32.700402,50.819868,3.026894,73.238571,0.727228,18.251563
std,7.548017e+06,37688.281473,37426.214911,1.771190e+05,798.060091,742.619512,24852.102298,200.486365,158.589205,538.232376,3.951055,3.025938,0.353535,2855.645548,26.918208,11727.079173,208.842214,555.994222,35.021798,10698.231091,212.961157,1.778689e+05,3.532861e+07,776.638003,6.025205,1.623359e+05,5.125479,0.097146,892.887426,1.915864e+08,1.073374e+08,5.398582e+07,3.293334e+06,2.245151e+06,32.723176,19.364008,14.768584,4535.058648,20.981550,6.790576e+08,1803.522029,9.130788,6.209365,4.235731,19997.918206,19.965054,119.072414,4.155569,10.466967,13.488450,31.758241,2.458076,7.556931,0.150349,35.848993
min,1.000000e+00,-74347.000000,-6223.000000,1.000000e+00,-1918.000000,-232.143000,0.001000,-2153.437000,-276.825000,0.001000,-76.445000,-10.921000,-0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.391720e+05,0.000000e+00,0.000000,-8.538000,0.000000e+00,0.000000,0.000000,1.100000,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,4.700000e+01,0.137000,15.100000,1.144000,0.526000,661.240000,0.100000,79.370000,0.990000,0.100000,7.700000,1.188000,0.100000,53.280000,0.394000,-95.590000
25%,1.337000e+03,2.000000,7.571000,5.600000e+01,0.000000,0.000000,268.655250,0.218000,1.307000,8.180250,0.000000,0.000000,0.840000,31.000000,4.749000,115.000000,22.910000,8.000000,1.805000,48.448000,9.294500,1.701000e+03,1.686005e+05,14.917500,0.148000,1.785000e+03,0.147000,0.016000,7.700000,1.336610e+05,1.045842e+05,5.392600e+04,4.689000e+03,8.810000e+0